In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
#load model
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # "unsloth/llama-3-8b-bnb-4bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head"],
    lora_alpha = 32,

    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,

    use_rslora = True,
    loftq_config = None,
)


Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [ ]:
from datasets import load_dataset

# Load your custom dataset
eye_qa_plus = load_dataset("QIAIUNCC/EYE-QA-PLUS", split="train[:2000]")  # Adjust the split as needed

# Split into train and test sets (80-20 split)
eye_qa_plus = eye_qa_plus.train_test_split(test_size=0.2)

README.md:   0%|          | 0.00/516 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/20.4M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.32M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28843 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4691 [00:00<?, ? examples/s]

In [ ]:
text = eye_qa_plus['train'][0]
text

{'input': 'Question: What is the relation between Biopsy of cornea (procedure) and Biopsia ?',
 'output': 'Biopsy of cornea (procedure) has method: Biopsia',
 'instruction': "Being an ophthalmologist, your task is to answer the medical questions based on the patient's description.\nDetermine the relation between the two medical terms.",
 'source': 'umls_relation'}

In [ ]:
from datasets import load_dataset
from unsloth import UnslothTrainer, UnslothTrainingArguments, FastLanguageModel
from transformers import DataCollatorForLanguageModeling

In [ ]:
# Load your custom dataset
eye_qa_plus = load_dataset("QIAIUNCC/EYE-QA-PLUS", split="train[:2000]")

# Split into train and test sets (80-20 split)
eye_qa_plus = eye_qa_plus.train_test_split(test_size=0.2)

# Preprocess
def preprocess_function(examples):
    inputs = examples["input"]  # Assuming this contains the context or instruction
    outputs = examples["output"]  # Assuming this contains the answer or output text
    instructions = examples["instruction"]

    # Format the input in a suitable way for your model
    input_text = [f"Instruction: {instruction} {input}" for instruction, input in zip(instructions, inputs)]
    target_text = outputs

    return {"input_text": input_text, "target_text": target_text}

# Apply preprocessing to the dataset
eye_qa_plus_processed = eye_qa_plus.map(preprocess_function, batched=True, num_proc=8, remove_columns=["input", "output", "instruction", "source"])

# Formatting with stop token
def formatting_func(example):
    # Return the formatted string as a dictionary
    return {"input_text": f"{example['input_text']} Answer: {example['target_text']}<|eot|>"}

# Apply formatting function
eye_qa_plus_processed = eye_qa_plus_processed.map(formatting_func, batched=False)

Map (num_proc=8):   0%|          | 0/1600 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
eye_qa_plus_processed["train"][0]
formatting_func(eye_qa_plus_processed["train"][1])

{'input_text': "Instruction: Your role as an ophthalmologist requires you to answer the medical questions taking into account the patient's description.\nAnalyze the question given its context. Give both long answer and yes/no decision. Question: Does whole exome sequencing identify CRB1 defect in an unusual maculopathy phenotype?\nContext: To report a new phenotype caused by mutations in the CRB1 gene in a family with 2 affected siblings. Molecular genetics and observational case studies. Two affected siblings and 3 unaffected family members. Each subject received a complete ophthalmic examination together with color fundus photography, fundus autofluorescence (FAF), and spectral-domain optical coherence tomography (SD-OCT). Microperimetry 1 (MP-1) mapping and electroretinogram (ERG) analysis were performed on the proband. Screening for disease-causing mutations was performed by whole exome sequencing in 3 family members followed by segregation analyses in the entire family. Appearanc

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #not using mask language

# Trainer
trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=eye_qa_plus_processed["train"],
    dataset_text_field="input_text",
    formatting_func=formatting_func,
    max_seq_length=512,
    data_collator=data_collator,
    args=UnslothTrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        warmup_ratio=0.1,
        num_train_epochs=3,
        learning_rate=5e-5,
        embedding_learning_rate=5e-6,
        fp16 = not is_bfloat16_supported(), # Use 16-bit floating point if bfloat16 isn't supported.
        bf16 = is_bfloat16_supported(),     #use bf16 if hardware support
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.00,
        lr_scheduler_type="cosine", #Cosine scheduler to adjust the learning rate over time.
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

# Train
trainer.train()

Unsloth: Tokenizing ["input_text"] (num_proc=12):   0%|          | 0/1600 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,600 | Num Epochs = 3 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 1,386,217,472/8,000,000,000 (17.33% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.947700
2,1.894900
3,2.164200
4,1.818200
5,2.331700
6,2.117900
7,1.944900
8,1.958900
9,1.868400
10,2.049100


TrainOutput(global_step=300, training_loss=1.0087445708115896, metrics={'train_runtime': 1064.0174, 'train_samples_per_second': 4.511, 'train_steps_per_second': 0.282, 'total_flos': 8.818770437318246e+16, 'train_loss': 1.0087445708115896})

In [ ]:
#saved the finetuned model
model.save_pretrained("outputs")
tokenizer.save_pretrained("outputs")

('outputs/tokenizer_config.json',
 'outputs/special_tokens_map.json',
 'outputs/tokenizer.json')

In [ ]:
!zip -r opticlarity_finetuned_model.zip outputs/

  adding: outputs/ (stored 0%)
  adding: outputs/checkpoint-300/ (stored 0%)
  adding: outputs/checkpoint-300/trainer_state.json (deflated 80%)
  adding: outputs/checkpoint-300/README.md (deflated 66%)
  adding: outputs/checkpoint-300/scheduler.pt (deflated 56%)
  adding: outputs/checkpoint-300/special_tokens_map.json (deflated 71%)
  adding: outputs/checkpoint-300/tokenizer_config.json (deflated 96%)
  adding: outputs/checkpoint-300/optimizer.pt (deflated 40%)
  adding: outputs/checkpoint-300/rng_state.pth (deflated 25%)
  adding: outputs/checkpoint-300/training_args.bin (deflated 51%)
  adding: outputs/checkpoint-300/adapter_config.json (deflated 55%)
  adding: outputs/checkpoint-300/adapter_model.safetensors (deflated 16%)
  adding: outputs/checkpoint-300/tokenizer.json (deflated 85%)
  adding: outputs/README.md (deflated 66%)
  adding: outputs/special_tokens_map.json (deflated 71%)
  adding: outputs/tokenizer_config.json (deflated 96%)
  adding: outputs/adapter_config.json (deflate

In [ ]:
from google.colab import files
files.download("opticlarity_finetuned_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>